# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [17]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display,update_display
from scraper import fetch_website_contents

In [2]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2'

In [5]:
# set up environment

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [9]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

systemMessage = """ You are coding assistant that takes a technical question,  
and responds with an explanation. """

Message=[{"role":"system","content": systemMessage},
         {"role":"user","content":question}]




In [18]:
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI()

def streamx():
    stream = openai.chat.completions.create(
        model=MODEL_GPT,
        messages=Message,
        stream=True
    )

    response = ""
    display_handle = display(Markdown(),display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response),display_id=display_handle.display_id)
        


    


In [19]:
streamx()

The line of code you've provided uses a generator expression along with `yield from`. Let's break it down to understand what it does and why it's structured this way.

1. **Understanding the Context**: 
   - It appears that `books` is presumably a collection, like a list or a set, where each element (`book`) is expected to be a dictionary that represents a book. Each book dictionary has various attributes, one of which is "author".

2. **Set Comprehension**: 
   - The expression `{book.get("author") for book in books if book.get("author")}` is a **set comprehension**. This creates a set of unique authors from the `books` collection.
   - `book.get("author")` retrieves the value associated with the key "author" for each `book`.
   - The condition `if book.get("author")` ensures that only books with non-None (or non-empty) authors are included, filtering out any books that do not have an "author" key present or whose value is falsy (like `None`, an empty string, etc.).

3. **Yield from**: 
   - The `yield from` statement in Python is used within a generator function to yield all values from an iterable. Here, it's used to yield each author from the set created by the comprehension. 
   - This means that rather than collecting authors into a list and then returning it, this code generates each author one at a time, which can save memory and can be more efficient for processing large datasets.

4. **Putting It All Together**: 
   - In summary, this line of code efficiently yields each unique author from the `books` collection without retaining the entire set in memory at once. As each author is yielded, it becomes immediately available to the caller of this generator function, allowing for iterative processing.

### Example Use Case
If `books` were defined as follows:

```python
books = [
    {"title": "Book 1", "author": "Author A"},
    {"title": "Book 2", "author": "Author B"},
    {"title": "Book 3", "author": "Author A"},
    {"title": "Book 4"},  # No author
    {"title": "Book 5", "author": None},  # No author
]
```

The code would yield:
- "Author A"
- "Author B"

Each time `next()` is called on the generator, the next unique author will be returned, reflecting those in the `books` list while automatically ignoring books with no authors.

In [20]:
# Get Llama 3.2 to answer
OLLAMA_BASE_URL = "http://localhost:11434/v1"
# Get gpt-4o-mini to answer, with streaming
openai = OpenAI(base_url=OLLAMA_BASE_URL,api_key="ollama")

def streamx2():
    stream = openai.chat.completions.create(
        model="llama3.2",
        messages=Message,
        stream=True
    )

    response = ""
    display_handle = display(Markdown(),display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        update_display(Markdown(response),display_id=display_handle.display_id)
        



In [21]:
streamx2()

**Explicative Code Explanation:**

```python
yield from {book.get("author") for book in books if book.get("author")}
```

This line of code uses Python's generator syntax and features to perform a concise data transformation.

Let's break it down:

- `yield from`: This keyword is used to delegate the execution of some generator to another. The yielded values are taken directly from the iterator passed to `yield from`.
- `{book.get("author") for book in books if book.get("author")}`: This is a dictionary comprehension, also known as a dictionary literal.
    - `for book in books`: This iterates over some collection of items (presumably named `books`), assigning each item to the variable `book`.
    - `if book.get("author")`: This filters out dictionaries where "author" key does not exist.

So overall, this line says: Yield all authors from the list of books. 

Here's a code example with an actual use case:

```python
books = [
    {"title": "Harry Potter", "author": "J.K. Rowling"},
    {"title": "The Lord of Rings", "author": "J.R.R. Tolkien"},
    {"title": "Game of Thrones",},
]

for author in yield from {book.get("author") for book in books if book.get("author")}: 
    print(author)
```

**Why?**

Using `yield from` is beneficial when we want to iterate over all items produced by an existing iterator, rather than using the usual generator expression or `function*` with a loop. 

`yield from` also has two advantages:
- It simplifies code and can reduce readability (as shown above).
- Since it's `yield` instead of `return`, it maintains the original function's state when the execution is resumed after the initial `yield`.

**Best Practice:**

Use this when you want to avoid unnecessary iterations in an existing complex loop. For instance:

```python
def print_complex():
    # Assume we start from a certain "complexity" as 5 for simplicity.
    def iterate(item, depth):
        print(indent*depth + item)
        
        for sub_item in items.get(item, []):
            if depth < complexity:
                yield from iterate(sub_item, depth+1)

    for key in complexities[:5]:
        yield from iterate(key, 0)

complexities = ["A", "B", "C"]
complexity = 4

# Result: 
for book in print_complex():
    pass
```